# 09 - Clock and Control Signals

## Introduction

The **clock** is the heartbeat of the CPU. It synchronizes all operations, ensuring that data moves through the system in an orderly fashion.

**Control signals** tell each component what to do on each clock cycle:
- Should the PC increment or load?
- Should memory read or write?
- Which ALU operation to perform?

## Learning Objectives

1. Understand clock timing and cycles
2. Learn about control signal organization
3. Build the Clock and ControlSignals classes

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root))

print("Setup complete!")

## Clock Timing

The clock alternates between high (1) and low (0):

```mermaid
sequenceDiagram
    participant CLK as Clock
    Note over CLK: Cycle 0
    CLK->>CLK: Rising Edge ↑
    Note over CLK: HIGH
    CLK->>CLK: Falling Edge ↓
    Note over CLK: Cycle 1
    CLK->>CLK: Rising Edge ↑
    Note over CLK: HIGH
    CLK->>CLK: Falling Edge ↓
    Note over CLK: Cycle 2
```

On each **rising edge** (0→1), edge-triggered flip-flops capture their inputs.

## Exercise 1: Clock Generator

In [ ]:
class Clock:
    """CPU Clock generator."""

    def __init__(self):
        self.cycle = 0
        self.state = 0  # 0 = low, 1 = high

    def tick(self) -> int:
        """Advance clock by one half-cycle.

        Returns:
            Current cycle number
        """
        # YOUR CODE HERE
        # Toggle state (0->1 or 1->0)
        # Increment cycle on falling edge (1->0)
        pass

    def reset(self) -> None:
        """Reset clock to initial state."""
        self.cycle = 0
        self.state = 0

    def get_state(self) -> int:
        """Get current clock state (0 or 1)."""
        return self.state


# Test
clk = Clock()
print("Clock ticks:")
for i in range(10):
    cycle = clk.tick()
    print(f"  Tick {i}: state={clk.get_state()}, cycle={cycle}")

## Control Signals

Control signals tell each component what to do. Here are the signals we need:

| Signal | Purpose |
|--------|---------|
| `pc_load` | Load new value into PC (for jumps) |
| `pc_inc` | Increment PC (normal execution) |
| `pc_reset` | Reset PC to 0 |
| `mem_read` | Read from memory |
| `mem_write` | Write to memory |
| `reg_write` | Write to register file |
| `alu_op` | 4-bit ALU operation code |
| `ir_load` | Load instruction register |
| `mem_to_reg` | Data comes from memory (not ALU) |

## Exercise 2: Control Signals Container

In [ ]:
class ControlSignals:
    """Container for all CPU control signals."""

    def __init__(self):
        # Program Counter
        self.pc_load = 0
        self.pc_inc = 0
        self.pc_reset = 0

        # Memory
        self.mem_read = 0
        self.mem_write = 0

        # Registers
        self.reg_write = 0
        self.reg_read_a = 0
        self.reg_read_b = 0

        # ALU
        self.alu_op = [0, 0, 0, 0]

        # Instruction Register
        self.ir_load = 0

        # Mux controls
        self.alu_src_b = 0  # 0=register, 1=immediate
        self.reg_dst = 0  # Which register to write
        self.mem_to_reg = 0  # 0=ALU result, 1=memory data

    def reset(self) -> None:
        """Reset all signals to 0."""
        self.__init__()

    def to_dict(self) -> dict:
        """Convert to dictionary for display."""
        return {
            "pc_load": self.pc_load,
            "pc_inc": self.pc_inc,
            "mem_read": self.mem_read,
            "mem_write": self.mem_write,
            "reg_write": self.reg_write,
            "alu_op": self.alu_op,
            "ir_load": self.ir_load,
            "mem_to_reg": self.mem_to_reg,
        }


# Test
signals = ControlSignals()
print("Default signals:")
for name, value in signals.to_dict().items():
    print(f"  {name}: {value}")

# Set up for a LOAD instruction
print("\nSignals for LOAD instruction:")
signals.mem_read = 1
signals.mem_to_reg = 1
signals.reg_write = 1
for name, value in signals.to_dict().items():
    if value != 0 and value != [0, 0, 0, 0]:
        print(f"  {name}: {value}")

## Control Signal Examples

Let's see what signals are needed for different instructions:

### FETCH (all instructions start here)
```
mem_read = 1   (read instruction from memory)
ir_load = 1    (store in instruction register)
```

### ADD Rd, Rs1, Rs2
```
alu_op = ADD   (select addition)
reg_write = 1  (write result to Rd)
pc_inc = 1     (go to next instruction)
```

### LOAD Rd, addr
```
mem_read = 1   (read from memory)
mem_to_reg = 1 (result comes from memory)
reg_write = 1  (write to register)
pc_inc = 1     (next instruction)
```

### JMP addr
```
pc_load = 1    (load new PC value)
```

In [ ]:
def signals_for_instruction(instruction: str) -> ControlSignals:
    """Generate control signals for an instruction (demo)."""
    signals = ControlSignals()

    if instruction == "FETCH":
        signals.mem_read = 1
        signals.ir_load = 1
    elif instruction == "ADD":
        signals.alu_op = [0, 0, 0, 0]  # ADD opcode
        signals.reg_write = 1
        signals.pc_inc = 1
    elif instruction == "LOAD":
        signals.mem_read = 1
        signals.mem_to_reg = 1
        signals.reg_write = 1
        signals.pc_inc = 1
    elif instruction == "STORE":
        signals.mem_write = 1
        signals.pc_inc = 1
    elif instruction == "JMP":
        signals.pc_load = 1

    return signals


# Display signals for each instruction type
for instr in ["FETCH", "ADD", "LOAD", "STORE", "JMP"]:
    print(f"\n{instr}:")
    sig = signals_for_instruction(instr)
    for name, value in sig.to_dict().items():
        if value != 0 and value != [0, 0, 0, 0]:
            print(f"  {name} = {value}")

## Copy Your Implementation

Once your code works, copy the `Clock` and `ControlSignals` classes to `src/computer/clock.py`

## Validation

In [ ]:
from utils.checker import check

check("clock")

## Summary

We've built the timing and control infrastructure:

| Component | Purpose |
|-----------|--------|
| Clock | Generates timing signal |
| ControlSignals | Container for all control lines |

### Key Concepts

1. **Clock** synchronizes all operations
2. **Rising edge** triggers state changes
3. **Control signals** tell components what to do
4. Each instruction requires specific signal combinations

### What's Next?

Now we need to define our **Instruction Set Architecture (ISA)** - the language our CPU speaks!